# OmicSynth NDD Drug Target Analysis

1. [Significant Genes](#siggen) <br>
2. [Summary Statistics](#sumstats) 
    1. [Number of Unique Genes in each Disease](#sumstats2.1)
    2. [Number of Unique Genes by eQTL](#sumstats2.2)
    3. [Number of Significant Unique Genes](#sumstats2.3)
        1. [p<sub>SMR_multi</sub> < 0.05 +  p<sub>HEIDI</sub> > 0.01](#sumstats2.3)
        2. [p<sub>SMR_multi</sub> < 0.05/16875 & p<sub>HEIDI</sub> > 0.01](#sumstats2.3.2)
        3. [p<sub>SMR_multi</sub> < 0.05/16875 & p<sub>HEIDI</sub> > 0.01](#sumstats2.3.2)
3. 

## Library Load and Data Read in

In [1]:
import pandas as pd
import subprocess
import sys
import numpy as np
import os
import glob
from scipy.stats import rankdata
#from omicsynth_func import *
import warnings
warnings.simplefilter(action='ignore')


# Display all rows from data frame using pandas
#pd.set_option('display.max_rows', None)

In [2]:
def extract_topsig_genes(main_df,  sig = 0.05):
    # create df to hold top genes
    top_gene_df = pd.DataFrame()
 
    df_adj = main_df.query(f'p_SMR_multi <= {sig} & p_HEIDI > 0.01')
    top_gene = df_adj[['Omic', 'Disease', 'Gene','probeID','ProbeChr', 'Probe_bp', 'topRSID','topSNP_chr', 'topSNP_bp', 'A1', 'A2', 'b_SMR', 'se_SMR','p_SMR', 'p_SMR_multi','p_HEIDI']] # filter out any genes whose fdr pval does not meet defined significance; default = 0.05

    top_gene_df = pd.concat([top_gene_df,top_gene])
    top_gene_df = top_gene_df.drop_duplicates()
        
    return top_gene_df

In [3]:
def extract_top_genes(main_df,dx_list, omic_list, pcol):
    
    # create df to hold top genes
    top_gene_df = pd.DataFrame()

    for disease in dx_list:
        for omic in omic_list:

            # filter for hits that have  FDR_pval < 0.05 and p_HEIDI > 0.01
            df = main_df.query(f"Disease == '{disease}' & Omic == '{omic}'")
            
            df = df.query("p_SMR_multi < 0.05 & p_HEIDI > 0.01")
            top_gene = df[df['p_SMR_multi'] == df['p_SMR_multi'].min()]
            top_gene = top_gene[['Omic', 'Disease', 'Gene','probeID','ProbeChr', 'Probe_bp', 'topRSID','topSNP_chr', 'topSNP_bp', 'A1', 'A2', 'b_SMR', 'se_SMR','p_SMR', 'p_SMR_multi','p_HEIDI']]

            top_gene_df = pd.concat([top_gene_df,top_gene])
    return top_gene_df

In [7]:
# load in SMR results for NDDs
print(os.getcwd())   #<-- This line new code as at 2023-11-23 18:48GMT (HD)
ndd_df = pd.read_csv("NDD_SMR_genes.csv")

# read in therapeutic drug data - from Finan et al and DGidb
drug_df = pd.read_csv('/data/CARD_AA/projects/omicSynth/v8/analysis/drug_genome_dgidb.csv', sep = ',')


# list of unique gene targets from drug data
thera_genes = list(drug_df['gene_name'].unique())


/media/hammie/My Passport/SOFTWARE/omicsynthcustomanalysis/NDD_SMR/scripts


FileNotFoundError: [Errno 2] No such file or directory: 'NDD_SMR_genes.csv'

In [ ]:
# read in protein coding genes
coding = pd.read_csv('proteincoding_genesym.csv')

# remove novel_or_none
coding = coding.query('Gene_symbol != "novel_or_none"')

# get list
cgenes = list(coding.Gene_symbol)

# print shape of df before removing non protein coding genes
ndd_df.shape

In [ ]:
# remove all non protein coding genes
ndd_df_pc = ndd_df.query('Gene == @cgenes')

# print shape of df after removing non protein coding genes
ndd_df_pc.shape

In [ ]:
# difference
dif = ndd_df.shape[0] - ndd_df_pc.shape[0]

print(f'Removed {dif} non protein coding genes')

In [ ]:
omic_map = {'Cerebellum_metaBrain': 'Cerebellum eQTL',
 'Basalganglia_metaBrain': 'Basal Ganglia eQTL',
 'Spinalcord_metaBrain': 'Spinalcord eQTL',
 'Hippocampus_metaBrain': 'Hippocampus eQTL',
 'brain_mMeta': 'Whole Brain meta-analysis mQTL',
 'blood_mcrae': 'Whole Blood mQTL',
 'Cortex_metaBrain': 'Cortex eQTL metaBrain',
 'Brain_Frontal_Cortex_BA9': 'Frontal Cortex BA9 eQTL',
 'Brain_Cerebellar_Hemisphere': 'Cerebellar Hemisphere eQTL',
 'psychEncode_prefrontal_cortex': 'Prefrontal Cortex eQTL',
 'Brain_Cortex': 'Cortex eQTL GTEx',
 'Brain_Caudate_basal_ganglia': 'Caudate Basal Ganglia eQTL',
 'Nerve_Tibial': 'Tibial Nerve eQTL',
 'Muscle_Skeletal': 'Skeletal Muscle eQTL',
 'Brain_Hippocampus': 'Hippocampus eQTL',
 'multiancestry': 'Multi Ancestry Whole Brain Meta-analysis eQTL',
 'Brain_Substantia_nigra': 'Substantia nigra eQTL',
 'atlas_csf': 'Cerebral Spinal Fluid pQTL',
 'Brain_Hypothalamus': 'Hypothalamus eQTL',
 'Liver': 'Liver eQTL',
 'Brain_Anterior_cingulate_cortex_BA24': 'Anterior Cingulate Cortex BA24 eQTL',
 'blood_eQTLgen': 'Whole Blood eQTL eQTLgen',
 'Brain_Putamen_basal_ganglia': 'Putamen Basal Ganglia eQTL',
 'Brain_Amygdala': 'Amygdala eQTL',
 'brain_eMeta': 'Whole Brain eQTL',
 'Whole_Blood': 'Whole Blood eQTL GTEx',
 'Brain_Cerebellum': 'Cerebellum eQTL',
 'Brain_Nucleus_accumbens_basal_ganglia': 'Nucleus Accumbens Basal Ganglia',
'Brain_Spinal_cord_cervical_c1': 'Cervical Spinal Cord C1 eQTL'}

In [ ]:
# change omics
ndd_df_pc['Omic Annotated'] = ndd_df_pc.Omic.map(omic_map)
ndd_df_pc = ndd_df_pc[['Omic Annotated', 'Disease', 'Gene', 'probeID', 'ProbeChr', 'Probe_bp', 'topRSID',
       'topSNP_chr', 'topSNP_bp', 'A1', 'A2', 'b_SMR', 'se_SMR', 'p_SMR',
       'p_SMR_multi', 'p_HEIDI']]
ndd_df_pc.rename({'Omic Annotated': 'Omic'}, axis = 1, inplace = True)
ndd_df_pc

In [ ]:
# get desired NDDs from Gtex
gtex_brain = glob.glob('./AD_Risk_Factors/GTEx/eQTL_besd_lite/Brain*')
gtex_liver = glob.glob('./AD_Risk_Factors/GTEx/eQTL_besd_lite/Liver*')
gtex_nerve = glob.glob('./AD_Risk_Factors/GTEx/eQTL_besd_lite/Nerve*')
gtex_muscle = glob.glob('./AD_Risk_Factors/GTEx/eQTL_besd_lite/Muscle*')
gtex_blood = glob.glob('./AD_Risk_Factors/GTEx/eQTL_besd_lite/*Blood*')

gtex_list = gtex_brain + gtex_liver + gtex_nerve + gtex_blood + gtex_muscle

gtex = []
for x in gtex_list:
    shortname = x.split('/')[-1].rsplit('.',2)[0] 
    if shortname not in gtex:
        gtex.append(x.split('/')[-1].rsplit('.',2)[0])
        
gtex = list(map(lambda x: x.replace('Brain_Spinal_cord_cervical_c-1', 'Brain_Spinal_cord_cervical_c1'), gtex))

ndd_list = ['AD', 'ALS', 'FTD', 'LBD', 'PD', 'PSP']

# add non-GTEx xQTL sources
ndd_omic = gtex + ['Cerebellum_metaBrain', 'Spinalcord_metaBrain', 'brain_eMeta', 'Cortex_metaBrain', 
            'Basalganglia_metaBrain', 'Hippocampus_metaBrain', 'blood_eQTLgen', 'brain_mMeta', 'blood_Bryois', 'blood_mcrae', 'psychEncode_prefrontal_cortex', 'atlas_csf', 'atlas_plasma', 'atlas_brain', 'multiancestry']

# list of eqtl omics except multiancestry
eqtl_omics = ['Brain_Amygdala', 'Brain_Hippocampus',
       'Brain_Anterior_cingulate_cortex_BA24',
       'Brain_Nucleus_accumbens_basal_ganglia', 'Brain_Hypothalamus',
       'Brain_Cerebellar_Hemisphere', 'Brain_Substantia_nigra',
       'Brain_Caudate_basal_ganglia', 'Brain_Putamen_basal_ganglia',
       'Brain_Cerebellum', 'Brain_Cortex', 'Brain_Frontal_Cortex_BA9',
       'Liver', 'Nerve_Tibial', 'Whole_Blood', 'Muscle_Skeletal',
       'Cerebellum_metaBrain', 'Spinalcord_metaBrain', 'brain_eMeta',
       'Cortex_metaBrain', 'Basalganglia_metaBrain',
       'Hippocampus_metaBrain', 'blood_eQTLgen', 'psychEncode_prefrontal_cortex', 'Brain_Spinal_cord_cervical_c1']

eqtl_omics2 = ['Cerebellum eQTL', 'Basal Ganglia eQTL', 'Spinalcord eQTL', 'Hippocampus eQTL', 'Cortex eQTL metaBrain', 'Frontal Cortex BA9 eQTL',
               'Cerebellar Hemisphere eQTL', 'Prefrontal Cortex eQTL', 'Cortex eQTL GTEx', 'Caudate Basal Ganglia eQTL', 'Tibial Nerve eQTL',
               'Skeletal Muscle eQTL', 'Hippocampus eQTL', 'Substantia nigra eQTL', 'Hypothalamus eQTL', 'Liver eQTL', 'Anterior Cingulate Cortex BA24 eQTL',
               'Whole Blood eQTL eQTLgen', 'Putamen Basal Ganglia eQTL', 'Amygdala eQTL', 'Whole Brain eQTL', 'Whole Blood eQTL GTEx', 'Cerebellum eQTL',
               'Nucleus Accumbens Basal Ganglia', 'Cervical Spinal Cord C1 eQTL']

## 1. Significant Genes <a id='siggen'></a>

### Significant Therapeutic Genes p_SMR_multi < 0.05/16875 & p_HEIDI > 0.01 - primary significance threshold for paper

In [ ]:
def extract_topsig_genes2(main_df, sig = 0.05): # no fdr just P SMR multi < 0.01/20000 and HEIDI > 0.01.
    # create df to hold top genes
    top_gene_df = pd.DataFrame()
    # pval based on largest number of unique protein coding genes amongst NDDs
    pval = sig/16875
    df = main_df.query(f"p_SMR_multi < {pval} & p_HEIDI > 0.01") # initial filter by each dx/omic combo  
    top_gene = df[['Omic', 'Disease', 'Gene','probeID','ProbeChr', 'Probe_bp', 'topRSID','topSNP_chr', 'topSNP_bp', 'A1', 'A2', 'b_SMR', 'se_SMR','p_SMR', 'p_SMR_multi','p_HEIDI']] # filter out any genes whose fdr pval does not meet defined significance; default = 0.05

    top_gene_df = pd.concat([top_gene_df,top_gene])
    top_gene_df = top_gene_df.drop_duplicates()
        
    return top_gene_df

In [ ]:
ndd_adj = extract_topsig_genes2(ndd_df_pc)

ndd_adj

In [ ]:
len(ndd_adj.Gene.unique()) # of significant genes

#### Per Disease Breakdown for significant hits at p_SMR_multi < 0.05/16875 & p_HEIDI > 0.01

In [ ]:
for dx in ndd_list:
    df = ndd_adj.query(f'Disease == "{dx}"')
    uni = len(df.Gene.unique())
    print(f'{dx} has {df.shape[0]} significant associations and {uni} unique genes')

In [ ]:
# grab most significant omic association for each gene and put into a df
top_genes_list_adj = ndd_adj.Gene.unique()
single_sig_adj = pd.DataFrame()
for gene in top_genes_list_adj:
    df = ndd_adj.query(f'Gene == "{gene}"')
    tmp = df[df['p_SMR_multi'] == df['p_SMR_multi'].min()]
    single_sig_adj = pd.concat([single_sig_adj, tmp])

# export single most significant omic association for each gene
#single_sig_adj[['Gene', 'Omic', 'Disease', 'topRSID', 'b_SMR','se_SMR', 'p_SMR', 'p_SMR_multi', 'p_HEIDI']].to_csv('top_sig_genetx_adj.csv', index = False)

In [ ]:
# find all diseases a gene is associated with
gene_dx_adj = pd.DataFrame()
dx_list_adj = []
for gene in top_genes_list_adj:
    df = ndd_adj.query(f'Gene == "{gene}"')
    
    # get list of diseases associated with a significant gene
    dxs = list(df.Disease.unique())
    
    # turn list into string to use as a column in output df
    dx_string = ', '.join(dxs)
    
    # append string to a list var 
    dx_list_adj.append(dx_string)

# create df
gene_dx_adj['Gene'] = top_genes_list_adj
gene_dx_adj['Diseases'] = dx_list_adj

# export
#gene_dx_adj.to_csv('top_gene_dx_combo_adj.csv', index = None)

In [ ]:
# final Gene-Disease-Omic combos
gene_dxtx_adj = pd.DataFrame()
tmp_list_adj = []

for gene in top_genes_list_adj:
    df = ndd_adj.query(f'Gene == "{gene}"')
    
    # create column that has every dx-omic combo for that gene
    df['dxtx'] = df.Disease + '-' + df.Omic
    #df = df.assign(dxtx=lambda x: str(x.Disease + '-' + x.Omic))
    
    # get list of each unique combo
    combos = list(set(df['dxtx'].unique()))
    
    # turn list into string to use as a column in output df
    dxtx_string = ', '.join(combos)
    
    # append string to a list var 
    tmp_list_adj.append(dxtx_string)
    
# create df
gene_dxtx_adj['Gene'] = top_genes_list_adj
gene_dxtx_adj['Diseases-Omic Associations'] = tmp_list_adj

#export
#gene_dxtx_adj.to_csv('gene_dxtx_combos_adj.csv', index = None)

In [ ]:
# final Gene-Disease-Omic combos and seperate by disease
gene_dxtx_adj = pd.DataFrame()
for gene in top_genes_list_adj:
    df = ndd_adj.query(f'Gene == "{gene}"')
    
    # groupby
    df2 = df.groupby(['Gene', 'Disease']).agg({'Gene':'first','Disease':'first', 'Omic': ', '.join})
  

    # append resulting dict into larger df for genes
    gene_dxtx_adj = pd.concat([gene_dxtx_adj, df2])

# export
#gene_dxtx_adj.to_csv('gene_dxtx_combos_adj.csv', index = None)
gene_dxtx_adj

### Genes Significant in 2 Diseases

In [ ]:
# grab most significant omic association for each gene and put into a df
top_genes_list_adj = ndd_adj.Gene.unique()
two_sig_adj = pd.DataFrame()
dx_2plus_adj = []
for gene in top_genes_list_adj:
    df = ndd_adj.query(f'Gene == "{gene}"')
    
    # get list of diseases associated with a significant gene
    dxs = list(df.Disease.unique())
    
    # only get info on genes that are isgnificant in 3+ dx

    if len(dxs) == 2:
        two_sig_adj = pd.concat([two_sig_adj, df])
        # append genes that meet criteria to a list
        dx_2plus_adj.append(gene)

# find all diseases a gene is associated with
gene_dx_adj2 = pd.DataFrame()
dx_list_adj2 = []
for gene in dx_2plus_adj:
    df = ndd_adj.query(f'Gene == "{gene}"')
    
    # get list of diseases associated with a significant gene
    dxs = list(df.Disease.unique())
    
    # turn list into string to use as a column in output df
    dx_string = ', '.join(dxs)
    
    # append string to a list var 
    dx_list_adj2.append(dx_string)

# create df
gene_dx_adj2['Gene'] = dx_2plus_adj
gene_dx_adj2['Diseases'] = dx_list_adj2

thera_sig_adj2 = []
non_sig_adj2 = []
for x in gene_dx_adj2['Gene']:
    if x in thera_genes: # thera_genes = list of thera genes from drug dbs
        thera_sig_adj2.append(x)
    else:
        non_sig_adj2.append(x)

In [ ]:
gene_dx_adj2.query('Gene == @thera_sig_adj2')

In [ ]:
gene_dx_adj2.query('Gene == @non_sig_adj2')  

In [ ]:
# use top gene list to pull all hits
dx2_df = ndd_adj.query('Gene == @dx_2plus_adj')
dx2_df

In [ ]:
# export 
dx2_df.to_csv('gene_2dx_hits.csv', index = None)

### Genes Significant in 3+ Diseases

In [ ]:
# grab most significant omic association for each gene and put into a df
top_genes_list_adj = ndd_adj.Gene.unique()
three_sig_adj = pd.DataFrame()
dx_3plus_adj = []
for gene in top_genes_list_adj:
    df = ndd_adj.query(f'Gene == "{gene}"')
    
    # get list of diseases associated with a significant gene
    dxs = list(df.Disease.unique())
    
    # only get info on genes that are isgnificant in 3+ dx

    if len(dxs) >= 3:
        three_sig_adj = pd.concat([three_sig_adj, df])
        # append genes that meet criteria to a list
        dx_3plus_adj.append(gene)

# find all diseases a gene is associated with
gene_dx_adj3 = pd.DataFrame()
dx_list_adj3 = []
for gene in dx_3plus_adj:
    df = ndd_adj.query(f'Gene == "{gene}"')
    
    # get list of diseases associated with a significant gene
    dxs = list(df.Disease.unique())
    
    # turn list into string to use as a column in output df
    dx_string = ', '.join(dxs)
    
    # append string to a list var 
    dx_list_adj3.append(dx_string)

# create df
gene_dx_adj3['Gene'] = dx_3plus_adj
gene_dx_adj3['Diseases'] = dx_list_adj3

thera_sig_adj3 = []
non_sig_adj3 = []
for x in gene_dx_adj3['Gene']:
    if x in thera_genes: # thera_genes = list of thera genes from drug dbs
        thera_sig_adj3.append(x)
    else:
        non_sig_adj3.append(x)

In [ ]:
gene_dx_adj3

In [ ]:
# use top gene list to pull all hits
dx3_df = ndd_adj.query('Gene == @dx_3plus_adj')
dx3_df

In [ ]:
# export 
dx3_df.to_csv('gene_3dx_hits.csv', index = None)

In [ ]:
gene_dx_adj3.query('Gene == @thera_sig_adj3') # druggable multidx genes

In [ ]:
gene_dx_adj3.query('Gene == @non_sig_adj3')  # nondruggable multidx genes

## 2. Summary Statistics <a id='sumstats'></a>

### 2.1 Number of Unique Genes in each Disease & Overall <a id='sumstats2.1'></a>

In [ ]:
# pull numbers from all genes used in SMR
for dx in ndd_list:
    df = ndd_df_pc.query(f"Disease == '{dx}'")
    num = len(df.Gene.unique())
    print(f'{dx} has {num} total genes')

In [ ]:
# pull numbers from all genes used in SMR
for dx in ndd_list:
    df = ndd_df_pc.query(f"Disease == '{dx}' & Omic == 'Liver eQTL'")
    num = len(df.Gene.unique())
    print(f'{dx} has {num} total liver genes')

In [ ]:
# extract therapuetic gene hits and non-therapeutic
ndd_thera = ndd_df_pc.query('Gene == @thera_genes')
ndd_nonthera = ndd_df_pc.query('Gene != @thera_genes')

In [ ]:
for dx in ndd_list:
    df = ndd_thera.query(f"Disease == '{dx}'")
    num = len(df.Gene.unique())
    print(f'{dx} has {num} therapeutic genes')

In [ ]:
for dx in ndd_list:
    df = ndd_nonthera.query(f"Disease == '{dx}'")
    num = len(df.Gene.unique())
    print(f'{dx} has {num} nontherapeutic genes')

### 2.2 eQTL  Stats<a id='sumstats2.2'></a>

#### All Genes - no sig threshold applied

In [ ]:
# filter out for all eQTL omics only
ndd_eQTL = ndd_df_pc.query('Omic == @eqtl_omics2')

multi_eqtl = ndd_df_pc.query('Omic == "Multi Ancestry Whole Brain Meta-analysis eQTL"')

ma_genes = list(multi_eqtl.Gene.unique()) # multiancestry gene hits

In [ ]:
# get number of eQTLS that aren't multiancestry for each disease
for dx in ndd_list:
    df = ndd_eQTL.query(f'Disease == "{dx}"')
    num = len(df.Gene.unique())
    print(f'There are {num} genes in {dx} eQTLs')

In [ ]:
# get number of eQTLS that are multiancestry for each disease
for dx in ndd_list:
    df = multi_eqtl.query(f'Disease == "{dx}"')
    num = len(df.Gene.unique())
    print(f'There are {num} unique genes in {dx} multiancestry eQTLs')

In [ ]:
# find how many hits replicated
for dx in ndd_list:
    eqtl_tmp = ndd_eQTL.query(f'Disease == "{dx}"')
    
    eqtl = eqtl_tmp.query("Gene == @ma_genes")
    
    num = len(eqtl.Gene.unique())
    print(f'There are {num} replicated multiancestry genes in {dx} eQTLs')

### 2.3 Significant genes <a id='sumstats2.3'></a>
#### 2.3.1 p<sub>SMR_multi</sub> < 0.05 +  p<sub>HEIDI</sub> > 0.01 

In [ ]:
# calculate how which genes are significant
all_sig = extract_topsig_genes(ndd_df_pc)

# pull numbers from significant genes used in SMR
for dx in ndd_list:
    df = all_sig.query(f"Disease == '{dx}'")
    num = len(df.Gene.unique())
    print(f'{dx} has {num} total unique genes')

In [ ]:
# pull numbers from significant genes used in SMR and split by therapeutic status
for dx in ndd_list:
    df = all_sig.query(f"Disease == '{dx}'")
    thera = df.query('Gene == @thera_genes')
    non_thera = df.query('Gene != @thera_genes')
    num1 = len(thera.Gene.unique())
    num2 = len(non_thera.Gene.unique())
    print(f'{dx} has {num1} thera genes and {num2} nonthera genes')

In [ ]:
# pull numbers for liver genes used in SMR
for dx in ndd_list:
    df = all_sig.query(f"Disease == '{dx}' & Omic == 'Liver eQTL'")
    num = len(df.Gene.unique())
    print(f'{dx} has {num} total unique liver genes')

In [ ]:
# filter out for all eQTL omics in first level of sig threshold
ndd_sigeQTL = all_sig.query('Omic == @eqtl_omics2')

multi_sigeqtl = all_sig.query('Omic == "Multi Ancestry Whole Brain Meta-analysis eQTL"')

In [ ]:
# get number of eQTLS that aren't multiancestry for each disease
for dx in ndd_list:
    df = ndd_sigeQTL.query(f'Disease == "{dx}"')
    num = len(df.Gene.unique())
    print(f'There are {num} genes in {dx} eQTLs')

In [ ]:
# get number of eQTLS that are multiancestry for each disease
for dx in ndd_list:
    df = multi_sigeqtl.query(f'Disease == "{dx}"')
    num = len(df.Gene.unique())
    print(f'There are {num} unique genes in {dx} multiancestry eQTLs')

In [ ]:
# find how many hits replicated using all multiancestry gene
# use list of all multiancestry eqtls(regardless of significance) to find any replicated hits in significant NDDs
for dx in ndd_list:
    eqtl_tmp = ndd_sigeQTL.query(f'Disease == "{dx}"')
    
    eqtl = eqtl_tmp.query("Gene == @ma_genes")
    
    num = len(eqtl.Gene.unique())
    print(f'There are {num} replicated multiancestry genes in {dx} eQTLs')

#### 2.3.2 p<sub>SMR_multi</sub> < 0.05/16875 & p<sub>HEIDI</sub> > 0.01 <a id='sumstats2.3.2'></a>

In [ ]:
sig2 = extract_topsig_genes2(ndd_df_pc) # pull main sig threshold hits (159 genes)

# filter out for all eQTL omics
ndd_sigeQTL = sig2.query('Omic == @eqtl_omics2')

multi_sigeqtl = sig2.query('Omic == "Multi Ancestry Whole Brain Meta-analysis eQTL"')

ma_genes = list(multi_sigeqtl.Gene.unique())

In [ ]:
# pull numbers from significant genes used in SMR
for dx in ndd_list:
    df = sig2.query(f"Disease == '{dx}'")
    num = len(df.Gene.unique())
    print(f'{dx} has {num} total unique genes')
    
# pull numbers from significant genes used in SMR and split by therapeutic status
for dx in ndd_list:
    df = sig2.query(f"Disease == '{dx}'")
    thera = df.query('Gene == @thera_genes')
    non_thera = df.query('Gene != @thera_genes')
    num1 = len(thera.Gene.unique())
    num2 = len(non_thera.Gene.unique())
    print(f'{dx} has {num1} thera genes and {num2} nonthera genes')

In [ ]:
# pull numbers for liver genes used in SMR
for dx in ndd_list:
    df = sig2.query(f"Disease == '{dx}' & Omic == 'Liver eQTL'")
    num = len(df.Gene.unique())
    print(f'{dx} has {num} total unique liver genes')

In [ ]:
# get number of eQTLS that aren't multiancestry for each disease
for dx in ndd_list:
    df = ndd_sigeQTL.query(f'Disease == "{dx}"')
    num = len(df.Gene.unique())
    print(f'There are {num} genes in {dx} eQTLs')

In [ ]:
# find how many hits replicated using all multiancestry gene
# use list of all multiancestry eqtls(regardless of significance) to find any replicated hits in significant NDDs
for dx in ndd_list:
    eqtl_tmp = ndd_sigeQTL.query(f'Disease == "{dx}"')
    
    eqtl = eqtl_tmp.query("Gene == @ma_genes")
    
    num = len(eqtl.Gene.unique())
    print(f'There are {num} replicated multiancestry genes in {dx} eQTLs')

#### 2.3.3 Significance p<sub>SMR_multi</sub> < 0.01/16875*[N Omic-Disease pairs]  & p<sub>HEIDI</sub> >  0.01 <a id='sumstats2.3.3'></a>

In [ ]:
def extract_topsig_genes3(main_df,dx_list, omic_list, sig = 0.05): # no fdr just P SMR multi < 0.01/20000 and HEIDI > 0.01.
    # create df to hold top genes
    top_gene_df = pd.DataFrame()
    all_df = main_df.query(f"Disease == '{dx_list[0]}'")
    txdx = len(all_df[['Omic', 'Disease']].value_counts()) # of omic-dx pairs for each disease
    pval = 0.05/(16875*txdx)
    for disease in dx_list:
        for omic in omic_list:
            df = main_df.query(f"Disease == '{disease}' & Omic == '{omic}' & p_SMR_multi < {pval} & p_HEIDI > 0.01") # initial filter by each dx/omic combo  
            top_gene = df[['Omic', 'Disease', 'Gene','probeID', 'topRSID', 'b_SMR', 'se_SMR','p_SMR', 'p_SMR_multi','p_HEIDI']] # filter out any genes whose fdr pval does not meet defined significance; default = 0.05

            top_gene_df = pd.concat([top_gene_df,top_gene])
            top_gene_df = top_gene_df.drop_duplicates()
        
    return top_gene_df

In [ ]:
sig3 = extract_topsig_genes3(ndd_df_pc, ndd_list, omic_map.values()) # pull hits that eet strictest threshold

# filter out for all eQTL omics
ndd_sigeQTL = sig3.query('Omic == @eqtl_omics2')

multi_sigeqtl = sig3.query('Omic == "Multi Ancestry Whole Brain Meta-analysis eQTL"')

ma_genes = list(multi_eqtl.Gene.unique())

In [ ]:
# pull numbers from significant genes used in SMR
for dx in ndd_list:
    df = sig3.query(f"Disease == '{dx}'")
    num = len(df.Gene.unique())
    print(f'{dx} has {num} total unique genes')
    
# pull numbers from significant genes used in SMR and split by therapeutic status
for dx in ndd_list:
    df = sig3.query(f"Disease == '{dx}'")
    thera = df.query('Gene == @thera_genes')
    non_thera = df.query('Gene != @thera_genes')
    num1 = len(thera.Gene.unique())
    num2 = len(non_thera.Gene.unique())
    print(f'{dx} has {num1} thera genes and {num2} nonthera genes')

In [ ]:
# pull numbers for liver genes used in SMR
for dx in ndd_list:
    df = sig3.query(f"Disease == '{dx}' & Omic == 'Liver eQTL'")
    num = len(df.Gene.unique())
    print(f'{dx} has {num} total unique liver genes')

In [ ]:
# get number of eQTLS that aren't multiancestry for each disease
for dx in ndd_list:
    df = ndd_sigeQTL.query(f'Disease == "{dx}"')
    num = len(df.Gene.unique())
    print(f'There are {num} genes in {dx} eQTLs')

In [ ]:
# find how many hits replicated using all multiancestry gene
# use list of all multiancestry eqtls(regardless of significance) to find any replicated hits in significant NDDs
for dx in ndd_list:
    eqtl_tmp = ndd_sigeQTL.query(f'Disease == "{dx}"')
    
    eqtl = eqtl_tmp.query("Gene == @ma_genes")
    
    num = len(eqtl.Gene.unique())
    print(f'There are {num} replicated multiancestry genes in {dx} eQTLs')

## 4. Tier Nomination <a id='tier'></a>

In [ ]:
# read in therapeutic drug data - from Finan et al and DGidb
drugs_df = pd.read_csv('./drug_genome_dgidb.csv', sep = ',')
# drop nans in gene_name
drugs_df = drugs_df.query('gene_name != "nan"')

# fill in any NaN bc theyre annoying
drugs_df.drug_concept_id = drugs_df.drug_concept_id.fillna('none')

# clean chemblid col since we need
drugs_df['chemblid'] = drugs_df.drug_concept_id.apply(lambda x: str(x.split(':')[1]) if ':'in x else x)


# remove any rows that do not have chembl id
drugs_df_red = drugs_df.query('chemblid != "none"')

drugs_df_red['drug_claim_primary_name'] = drugs_df_red['drug_claim_primary_name'].astype('str')

drugs_df_red['drug_claim_primary_name'] = drugs_df_red['drug_claim_primary_name'].apply(lambda x: x.lower())

In [ ]:
# pull genes that have medications
drugs_df_no = drugs_df.query('drug_name ! = "none"')

# list of unique gene targets from drug data
thera_genes = list(drugs_df_no['gene_name'].unique())

drugs_df_no

In [ ]:
# extract all genes that meet significance, not just top single gene ( only need to run this cell if you didn't run it in sections before)
ndd_adj = extract_topsig_genes2(ndd_df_pc)

# number of unique genes that are significant at p < 0.05/16875 & p_HEIDI > 0.01
print(len(ndd_adj.Gene.unique()))

### 4.1 Tier 1 - Novel Genes Multi Ancestry p_SMR_multi < 0.05/16875 and p_HEIDI > 0.01<a id='eqtl4.1'></a>

In [ ]:
# split data frame into therapeutic vs non-therapeutic
therapeutic_genes = ndd_adj.query('Gene == @thera_genes')
nonthera_genes = ndd_adj.query('Gene != @thera_genes')

# split by multiancestry
# filter for all non-multiancestry eQTLs
ndd_other = therapeutic_genes.query("Omic != 'Multi Ancestry Whole Brain Meta-analysis eQTL'")

# filter for multiancestry
ndd_ma = therapeutic_genes.query("Omic == 'Multi Ancestry Whole Brain Meta-analysis eQTL'")

# extract list of genes from multiancestry hits to compare against all other eQTLs
ma_genes = list(ndd_ma['Gene'].unique())
print(len(ma_genes))

other_genes = list(ndd_other.Gene.unique())
print(len(other_genes))

common_genes = []

for x in ma_genes:
    if x in other_genes:
        common_genes.append(x)
len(common_genes)

In [ ]:
# pull any hits from all other eqtls that passed inital filter
merge_hits_other = ndd_other.query("Gene == @common_genes")
merge_hits_ma = ndd_ma.query("Gene == @common_genes")

# combine ma hits df and merge_hits
merged_hits_all = pd.concat([merge_hits_other, merge_hits_ma])


# merge results with drug data
merged_drugs = merged_hits_all.merge(drugs_df[['gene_name', 'interaction_types', 'drug_claim_name', 'drug_claim_primary_name', 'drug_name', 'chemblid']], left_on = 'Gene', right_on = 'gene_name')

# sig genes replicated in MA
len(merged_drugs.Gene.unique())

In [ ]:
# find drugs with chemblid's and export for opentargets API
chembl_drugs = merged_drugs.query('chemblid != "none"')

# export df
#chembl_drugs.to_csv('sig_genes_ma_chemblthresh2.csv', index = None)

# find drugs with no chemblid's and export 
nochembl_drugs = merged_drugs.query('chemblid == "none"')

# export df
#nochembl_drugs.to_csv('sig_genes_ma_nochemblthresh2.csv', index = None)

### Filter for genes that have no known therapeutic drug

In [ ]:
# list of genes with drugs 
drug_genes = list(drugs_df_no['gene_name'].unique())

# filter SMR hits identified against ones with known thera
t1_novel = merged_hits_all.query('Gene != @drug_genes')

t1_novel.shape

### 4.2 T1 Novel - all Significant hits regardless of multiancestry replication

In [ ]:
# split data frame into therapeutic vs non-therapeutic
therapeutic_genes = ndd_adj.query('Gene == @thera_genes')
nonthera_genes = ndd_adj.query('Gene != @thera_genes')

print(len(therapeutic_genes.Gene.unique()))

# list of genes with drugs 
drug_genes = list(drugs_df['gene_name'].unique())
# filter SMR hits identified against ones with known thera
t1_novel = therapeutic_genes.query('Gene != @drug_genes')

t1_novel

In [ ]:
# merge thera genes on drug data
thera_drug_data = therapeutic_genes.merge(drugs_df[['gene_name', 'interaction_types', 'drug_claim_name', 'drug_claim_primary_name', 'drug_name', 'chemblid']], left_on = 'Gene', right_on = 'gene_name')

# find drugs with chemblid's and export for opentargets API
chembl_drugs_all = thera_drug_data.query('chemblid != "none"')

len(chembl_drugs_all.Gene.unique())

In [ ]:
len(thera_drug_data.Gene.unique()) - 3 # 3 being 3 tier 1 genes we identified using opentargets

In [ ]:
# export df
#chembl_drugs_all.to_csv('all_sig_genes_chemblthresh2.csv', index = None)

# find drugs with no chemblid's and export
nochembl_drugs_all = thera_drug_data.query('chemblid == "none"')
nochembl_drugs_all

len(nochembl_drugs_all.Gene.unique())

# export df
#nochembl_drugs_all.to_csv('all_sig_genes_nochemblthresh2.csv', index = None)

### Tier 3 - No Therapeutic Status

In [ ]:
nonthera_genes

In [ ]:
# number of unique genes
len(nonthera_genes.Gene.unique())